In [1]:
import os
import time
import numpy as np
import pyfftw as ft
import matplotlib.pyplot as plt
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB
from scipy.interpolate import RegularGridInterpolator

In [2]:
N = 1024
Nx = isotropic1024coarse['nx']; Lx = isotropic1024coarse['lx']
Ny = isotropic1024coarse['ny']; Ly = isotropic1024coarse['ly']
Nz = isotropic1024coarse['nz']; Lz = isotropic1024coarse['lz']

In [3]:
start = time.time()

u = np.zeros((N,N,N), dtype='float32')
v = np.zeros((N,N,N), dtype='float32')
w = np.zeros((N,N,N), dtype='float32')

end = time.time()
print(end-start)

0.000752210617065


In [4]:
start = time.time()

nproc = 8
nx = N//nproc

vx = np.zeros((N//nproc,N,N), dtype='float32')
vy = np.zeros((N//nproc,N,N), dtype='float32')
vz = np.zeros((N//nproc,N,N), dtype='float32')

folder = '/home/idies/workspace/scratch'

for k in range(nproc):
    filename = 'ref-isotropic1024coarse-'+str(k)+'-(t=0.0).npz'
    file = folder+'/'+filename
    
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    
    vx[:,:,:] = content['vx']
    vy[:,:,:] = content['vy']
    vz[:,:,:] = content['vz']
    
    u[k*nx:(k+1)*nx,:,:] = vx[:,:,:]
    v[k*nx:(k+1)*nx,:,:] = vy[:,:,:]
    w[k*nx:(k+1)*nx,:,:] = vz[:,:,:]

del vx,vy,vz
    
end = time.time()
print(end-start)

211.436427832


In [5]:
u2avg = np.average(u**2)
v2avg = np.average(v**2)
w2avg = np.average(w**2)

print(u2avg,v2avg,w2avg)

(0.41078633, 0.57521385, 0.37943667)


In [6]:
u2_sum = (u2avg + v2avg + w2avg)/3

print(np.sqrt(u2_sum))

0.674644793545


In [26]:
avg = u2avg + v2avg + w2avg
print(avg)

1.36544


In [30]:
start = time.time()

df2lt = np.zeros((Nx//2,3),dtype='float32')
for incr in range(Nx//2):
    if(incr%16==0):
        print("increment = ",incr)
        
    #############################
    
    df2avx = 0.
    for k in range(Nx):
        kprime = (k+incr)%Nx
        
        df = u[kprime,:,:] - u[k,:,:]
        
        df2slab = np.average(df**2)
        df2avx += df2slab
        
    df2avx /= Nx
        
    #############################
        
    df2avy = 0.
    for k in range(Ny):
        kprime = (k+incr)%Ny
        
        df = v[:,kprime,:] - v[:,k,:]
        
        df2slab = np.average(df**2)
        df2avy += df2slab
        
    df2avy /= Ny
        
    #############################        
        
    df2avz = 0.
    for k in range(Nz):
        kprime = (k+incr)%Nz
        
        df = w[:,:,kprime] - w[:,:,k]
        
        df2slab = np.average(df**2)
        df2avz += df2slab
        
    df2avz /= Nz
        
    #############################        
    
    df2lt[incr][0] = df2avx
    df2lt[incr][1] = df2avy
    df2lt[incr][2] = df2avz
    
end = time.time()
print(end-start)

('increment = ', 0)
('increment = ', 16)
('increment = ', 32)
('increment = ', 48)
('increment = ', 64)
('increment = ', 80)
('increment = ', 96)
('increment = ', 112)
('increment = ', 128)
('increment = ', 144)
('increment = ', 160)
('increment = ', 176)
('increment = ', 192)
('increment = ', 208)
('increment = ', 224)
('increment = ', 240)
('increment = ', 256)
('increment = ', 272)
('increment = ', 288)
('increment = ', 304)
('increment = ', 320)
('increment = ', 336)
('increment = ', 352)
('increment = ', 368)
('increment = ', 384)
('increment = ', 400)
('increment = ', 416)
('increment = ', 432)
('increment = ', 448)
('increment = ', 464)
('increment = ', 480)
('increment = ', 496)
47163.5811098


In [ ]:
r = np.linspace(0,np.pi,num=Nx//2,endpoint=True)

filename = 'direct_longitudinal_sf_u.npz'

np.savez(filename,df2lt=df2lt,avg = avg, r=r)

In [ ]:
start = time.time()

df2tv = np.zeros((Nx//2,3),dtype='float32')
for incr in range(Nx//2):
    if(incr%16==0):
        print("increment = ",incr)
        
    #############################
    
    df2avx = 0.
    for k in range(Nx):
        kprime = (k+incr)%Nx
        
        df = v[kprime,:,:] - v[k,:,:]
        
        df2slab = np.average(df**2)
        df2avx += df2slab
        
        ############
        
        df = w[kprime,:,:] - w[k,:,:]
        
        df2slab = np.average(df**2)
        df2avx += df2slab
        
    df2avx /= Nx
        
    #############################
        
    df2avy = 0.
    for k in range(Ny):
        kprime = (k+incr)%Ny
        
        df = u[:,kprime,:] - u[:,k,:]
        
        df2slab = np.average(df**2)
        df2avy += df2slab
        
        ############
        
        df = w[:,kprime,:] - w[:,k,:]
        
        df2slab = np.average(df**2)
        df2avy += df2slab
        
    df2avy /= Ny
        
    #############################
        
    df2avz = 0.
    for k in range(Nz):
        kprime = (k+incr)%Nz
        
        df = u[:,:,kprime] - u[:,:,k]
        
        df2slab = np.average(df**2)
        df2avz += df2slab
        
        ###########
        
        df = v[:,:,kprime] - v[:,:,k]
        
        df2slab = np.average(df**2)
        df2avz += df2slab
        
    df2avz /= Nz
        
    #############################        
    
    df2tv[incr][0] = df2avx
    df2tv[incr][1] = df2avy
    df2tv[incr][2] = df2avz
    
end = time.time()
print(end-start)

('increment = ', 0)


In [ ]:
r = np.linspace(0,np.pi,num=Nx//2,endpoint=True)

filename = 'direct_transverse_sf_u.npz'

np.savez(filename,df2tv=df2tv,avg = avg, r=r)